In [ ]:
from collections import OrderedDict, defaultdict
from IPython.display import Latex, display
import matplotlib.pyplot as plt
import sympy
from chempy import Substance, Equilibrium, Reaction, ReactionSystem
from chempy.kinetics.ode import get_odesys
from chempy.kinetics.rates import MassAction
from chempy.thermodynamics.expressions import EqExpr
from chempy.util.graph import rsys2graph
from chempy.util.pyutil import defaultkeydict
%matplotlib inline

In [ ]:
substances = OrderedDict([
    ('N', Substance('N', composition={'protein': 1}, latex_name='[N]')),
    ('U', Substance('U', composition={'protein': 1}, latex_name='[U]')),
    ('A', Substance('A', composition={'protein': 1}, latex_name='[A]')),
    ('L', Substance('L', composition={'ligand': 1}, latex_name='[L]')),
    ('NL', Substance('NL', composition={'protein': 1, 'ligand': 1}, latex_name='[NL]')),
])

In [ ]:
def _gibbs(args, T, R, backend):
    H, S, Cp, Tref = args
    H2 = H + Cp*(T - Tref)
    S2 = S + Cp*backend.log(T/Tref)
    return backend.exp(-(H2 - T*S2)/(R*T))

def _eyring(args, T, R, k_B, h, backend):
    H, S = args
    return k_B/h*T*backend.exp(-(H - T*S)/(R*T))

In [ ]:
Gibbs = EqExpr.from_callback(_gibbs, parameter_keys=('temperature', 'R'), argument_names=('H', 'S', 'Cp', 'Tref'))
Eyring = MassAction.from_callback(_eyring, parameter_keys=('temperature', 'R', 'k_B', 'h'), argument_names=('H', 'S'))

In [ ]:
thermo_dis = Gibbs(unique_keys=('He_dis', 'Se_dis', 'Cp_dis', 'Tref_dis'))
thermo_u = Gibbs(unique_keys=('He_u', 'Se_u', 'Cp_u', 'Tref_u'))  # ([He_u_R, Se_u_R, Cp_u_R, Tref])
kinetics_agg = Eyring(unique_keys=('Ha_agg', 'Sa_agg'))  # EyringMassAction([Ha_agg, Sa_agg])
kinetics_as = Eyring(unique_keys=('Ha_as', 'Sa_as'))
kinetics_f = Eyring(unique_keys=('Ha_f', 'Sa_f'))

In [ ]:
eq_dis = Equilibrium({'NL'}, {'N', 'L'}, thermo_dis, name='ligand-protein dissociation')
eq_u = Equilibrium({'N'}, {'U'}, thermo_u, {'L'}, {'L'}, name='protein unfolding')
r_agg = Reaction({'U'}, {'A'}, kinetics_agg, {'L'}, {'L'}, name='protein aggregation')

In [ ]:
rsys = ReactionSystem(
    eq_dis.as_reactions(kb=kinetics_as, new_name='ligand-protein association') +
    eq_u.as_reactions(kb=kinetics_f, new_name='protein folding') +
    (r_agg,), substances, name='4-state CETSA system')
rsys

In [ ]:
vecs, comp = rsys.composition_balance_vectors()
names = rsys.substance_names()
dict(zip(comp, [dict(zip(names, v)) for v in vecs]))

In [ ]:
rsys2graph(rsys, '4state.png', save='.', include_inactive=False)
from IPython.display import Image; Image('4state.png')

In [ ]:
from IPython.display import HTML
HTML('1st order processes\n' + rsys.unimolecular_html_table()[0] +
     '<br><br>2nd order processes\n' + rsys.bimolecular_html_table()[0])

In [ ]:
def mk_Symbol(key):
    if key in substances:
        arg = substances[key].latex_name
    else:
        arg = key.replace('temperature', 'T')
    return sympy.Symbol(arg)

autosymbols = defaultkeydict(mk_Symbol)
rnames = {}
for rxn in rsys:
    rnames[rxn.name] = rxn.name.replace(' ', '~').replace('-','-')
    rate_expr_str = sympy.latex(rxn.rate_expr()(autosymbols, backend=sympy))
    lstr = r'$r(\mathrm{%s}) = %s$' % (rnames[rxn.name], rate_expr_str)
    display(Latex(lstr))

In [ ]:
k = 'protein unfolding'
assert isinstance(rsys[k].param, MassAction)
assert rsys[k].param.rxn == rsys[k]
rsys[k].rate_expr()(defaultkeydict(sympy.Symbol), backend=sympy)

In [ ]:
type(rsys[k].param)

In [ ]:
for rxn in rsys:
    print(rxn.rate_expr()(autosymbols, backend=sympy))

In [ ]:
ratexs = [autosymbols['r(\mathrm{%s})' % rnames[rxn.name]] for rxn in rsys.rxns]
rates = rsys.rates(autosymbols, backend=sympy, ratexs=ratexs)
for k, v in rates.items():
    display(Latex(r'$\frac{[%s]}{dt} = %s$' % (k, sympy.latex(v))))

In [ ]:
odesys = get_odesys(rsys, include_params=False)

In [ ]:
params = dict(
    R = 8.314472,  # or N_A & kB
    kB = 1.3806504e-23,
    h = 6.62606896e-34,  # kB/h == 2.083664399411865e10 K**-1 * s**-1
    Ha_as = 30e3,
    Sa_as = -200,
    He_dis = 
)

In [ ]:
c0 = defaultdict(float, {'N': 1, 'L': 10})
odesys.integrate(1e10, c0, params, integrator='cvode', nsteps=2000)